In [5]:
import os
import json
import numpy as np
from omegaconf import OmegaConf

In [6]:
exp_dir = "../../../logs/en/asr/evaluate"
folders = {
    "AC_1": "version_24",
    "AC_100": "version_32",
    "AC_10": "version_33",
    "AC_0.1": "version_34",
    "AC_0.01": "version_35",
}

In [7]:
# define which accent is standard, which are seen and which are unseen
classes = {
    "seen": ["us", "uk", "ca", "in", "de", "au"],
    "unseen": ["hk", "sg", "sc", "nz", "ie", "za", "ni", "ph"],
}

In [8]:
# load WERs of each file, for each accent
wers = dict()
for exp, folder in folders.items():
    wers[exp] = {k[5:-4]: v for k, v in json.load(open(f"{exp_dir}/{folder}/avg_wers.json")).items()}

In [9]:
# transform dictionary into a numpy array, where one axis represents the accents and another the experiments
experiments = list(wers.keys())
accents = list(wers[experiments[0]].keys())
n_words = np.array([wers[experiments[0]][acc]["n_words"] for acc in accents])
avg_wers = np.array([[wers[exp][acc]["avg_wer"] for exp in experiments] for acc in accents])

In [10]:
# compute means for each class, for each experiment
class_indices = {k: [accents.index(acc) for acc in v] for k, v in classes.items()}
class_means = {k: np.mean(avg_wers[indices], axis=0) for k, indices in class_indices.items()}

In [11]:
# print the avg. WERs and the means (overall and per class) as a markdown table
headers = ["Accent / Dialect"] + experiments
print((" | ").join(headers))
print(f"|{'---:|'*len(headers)}")

# print avg. WERs
for i in range(len(accents)):
    row = [accents[i]] + [f"{avg_wers[i,j]*100:.2f}" for j in range(len(experiments))]
    print((" | ").join(row))

# print overall means
row = ["mean"] + [f"{np.mean(avg_wers[:,i])*100:.2f}" for i in range(len(experiments))]
print((" | ").join(row))

# print class means
for key, value in class_means.items():
    row = [f"{key} mean"] + [f"{value[i]*100:.2f}" for i in range(len(experiments))]
    print((" | ").join(row))

# print worst avg. WER of each experiment
row = ["worst"] + [f"{v*100:.2f}" for v in np.max(avg_wers, axis=0)]
print((" | ").join(row))

# print the avg. of the three worst avg. WERs of each experiment
row = ["3-worst mean."] + [f"{np.mean(np.sort(avg_wers, axis=0)[-3:,i])*100:.2f}" for i in range(len(experiments))]
print((" |s ").join(row))


Accent / Dialect | AC_1 | AC_100 | AC_10 | AC_0.1 | AC_0.01
|---:|---:|---:|---:|---:|---:|
za | 6.54 | 8.50 | 6.61 | 6.54 | 6.54
sg | 10.51 | 12.37 | 10.48 | 10.36 | 10.38
in | 9.41 | 12.02 | 9.38 | 9.21 | 9.19
au | 7.53 | 9.68 | 7.56 | 7.46 | 7.44
hk | 9.75 | 11.22 | 9.77 | 9.73 | 9.71
ie | 7.41 | 8.99 | 7.46 | 7.47 | 7.45
uk | 5.87 | 7.11 | 5.85 | 5.74 | 5.75
ph | 8.79 | 10.38 | 8.82 | 8.80 | 8.78
de | 5.66 | 7.76 | 5.58 | 5.56 | 5.57
ni | 7.02 | 8.13 | 7.04 | 6.94 | 6.93
us | 5.46 | 7.41 | 5.45 | 5.40 | 5.41
ca | 5.35 | 6.93 | 5.34 | 5.31 | 5.32
nz | 5.28 | 7.07 | 5.27 | 5.33 | 5.35
sc | 32.55 | 36.38 | 32.88 | 33.80 | 33.60
mean | 9.08 | 11.00 | 9.11 | 9.12 | 9.10
seen mean | 6.55 | 8.49 | 6.53 | 6.44 | 6.45
unseen mean | 10.98 | 12.88 | 11.04 | 11.12 | 11.09
worst | 32.55 | 36.38 | 32.88 | 33.80 | 33.60
3-worst mean. |s 17.61 |s 20.26 |s 17.71 |s 17.96 |s 17.90


Accent / Dialect | AC_1 | AC_100 | AC_10 | AC_0.1 | AC_0.01
|---:|---:|---:|---:|---:|---:|
za | 6.54 | 8.50 | 6.61 | 6.54 | 6.54
sg | 10.51 | 12.37 | 10.48 | 10.36 | 10.38
in | 9.41 | 12.02 | 9.38 | 9.21 | 9.19
au | 7.53 | 9.68 | 7.56 | 7.46 | 7.44
hk | 9.75 | 11.22 | 9.77 | 9.73 | 9.71
ie | 7.41 | 8.99 | 7.46 | 7.47 | 7.45
uk | 5.87 | 7.11 | 5.85 | 5.74 | 5.75
ph | 8.79 | 10.38 | 8.82 | 8.80 | 8.78
de | 5.66 | 7.76 | 5.58 | 5.56 | 5.57
ni | 7.02 | 8.13 | 7.04 | 6.94 | 6.93
us | 5.46 | 7.41 | 5.45 | 5.40 | 5.41
ca | 5.35 | 6.93 | 5.34 | 5.31 | 5.32
nz | 5.28 | 7.07 | 5.27 | 5.33 | 5.35
sc | 32.55 | 36.38 | 32.88 | 33.80 | 33.60
mean | 9.08 | 11.00 | 9.11 | 9.12 | 9.10
seen mean | 6.55 | 8.49 | 6.53 | 6.44 | 6.45
unseen mean | 10.98 | 12.88 | 11.04 | 11.12 | 11.09
worst | 32.55 | 36.38 | 32.88 | 33.80 | 33.60
3-worst mean. | 17.61 | 20.26 | 17.71 | 17.96 | 17.90

In [12]:
# print the experiment folders of each experiment (both train and eval folders)
print("Experiment folders:\n")
for exp, folder in folders.items():
    eval_folder = os.path.join(exp_dir, folder)
    eval_config = OmegaConf.load(os.path.join(eval_folder, "config.yaml"))
    train_folder = f'../{eval_config.asr.ckpt.replace("/checkpoints/last.ckpt", "")}'
    print(f"- {exp}: training `{train_folder[3:]}`, evaluation `{eval_folder[3:]}`")

Experiment folders:

- AC_1: training `logs/en/ensemble/train/binary/b7/DAT/version_66`, evaluation `../../logs/en/asr/evaluate/version_24`
- AC_100: training `logs/en/ensemble/train/binary/b7/DAT/version_74`, evaluation `../../logs/en/asr/evaluate/version_32`
- AC_10: training `logs/en/ensemble/train/binary/b7/DAT/version_75`, evaluation `../../logs/en/asr/evaluate/version_33`
- AC_0.1: training `logs/en/ensemble/train/binary/b7/DAT/version_76`, evaluation `../../logs/en/asr/evaluate/version_34`
- AC_0.01: training `logs/en/ensemble/train/binary/b7/DAT/version_77`, evaluation `../../logs/en/asr/evaluate/version_35`
